# Implementing and fitting a simple syrinx model

Model based on [Mindlin et al., 2003](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.68.041908). 

We want to fit the system of first-order nonlinear equations:

$$
\dot{x} = y \\
\dot{y} = -\epsilon x - C x^2 y + By - D_0
$$
where
$$
\epsilon = \epsilon_1 + \epsilon_2 K(t) \\
B = \beta_1 + \beta_2 P(t) \\
D_0 = \delta D(t)
$$
and $K(t)$, $D(t)$, and $P(t)$ are the (linear envelopes of) tension in the ventral syringeal muscle (vS), the tracheobronchialis dorsalis (dTB), and sub-syringeal air pressure, respectively.

From the original paper, we take parameter values
$$
\epsilon_1 = 1.25 \times 10^8 \, \mathrm{s}^{-2} \\
\epsilon_2 = 7.5 \times 10^9 \, \mathrm{V}^{-1}\cdot \mathrm{s}^{-2} \\
C = 2 \times 10^8 \, \mathrm{cm}^{-2} \cdot \mathrm{s}^{-1} \\
\beta_1 = -2 \times 10^3 \, \mathrm{s}^{-1} \\
\beta_2 = 5.3 \times 10^4 \, \mathrm{V}^{-1}\cdot \mathrm{s}^{-1} \\
\delta = 15 \times 10^6 \, \mathrm{cm}\cdot\mathrm{V}^{-1} \cdot \mathrm{s}^{-2}
$$

In [ ]:
eps1 = 1.25e8
eps2 = 7.5e9
C = 2e8
beta1 = -2e3
beta2 = 5.3e4
delta = 15e6

But whereas the original paper used measured values for $K$, $P$, and $D$, we use simpler time series constructed to have the same shape:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

In [ ]:
sr = 4e4  # data sampling rate (Hz)
T = 2
t_axis = np.arange(0, T, 1/sr)

In [ ]:
def make_pulse_sequence(fn, locs):
    return lambda t: np.sum([fn(t, loc) for loc in locs])

In [ ]:
def make_pressure_pulse(loc, freq, amp, cutoff, dc):
    fn = lambda t: amp * np.maximum(np.sin(2 * np.pi * freq * (t - loc)), cutoff) + dc 
    return fn

pfreq = 2  # pressure pulse frequency (Hz)
pA = 0.015  # pressure amplitude (Volts)
p0 = 0.0  # pressure DC offset (Volts)
pcutoff = -.7
ploc = 3/16
P = make_pressure_pulse(ploc, pfreq, pA, pcutoff, p0)

plt.plot(t_axis, P(t_axis))

In [ ]:
def make_tension_pulse_fn(shape=1, scale=1, peak=1):
    norm = peak * np.exp(shape - shape * np.log(shape) - shape * np.log(scale)) 

    fn = lambda t, loc: norm * np.exp((t - loc)/scale) * np.maximum((loc - t), 0)**shape 

    return fn
    
        
kshape = 5  # shape parameter of gamma function
kscale = 0.025  # rate parameter of gamma function (s)
kpeak = 0.06  # peak value (Volts)


pulse = make_tension_pulse_fn(shape=kshape, scale=kscale, peak=kpeak)
K = np.vectorize(make_pulse_sequence(pulse, [0, 0.5, 1, 1.5]))


In [ ]:
dfreq = 2 
dlocs1 = np.arange(0.1, t_axis[-1], 1/dfreq)  # pressure pulse frequency (Hz)
dlocs2 = np.arange(0.45, t_axis[-1], 1/dfreq)  # pressure pulse frequency (Hz)
dlocs = np.sort(np.concatenate([dlocs1, dlocs2]))
dA = 0.05
dwid = 0.01

gpulse = lambda t, loc=0: dA * np.exp(-0.5 * (t - loc)**2/dwid**2)

D = np.vectorize(make_pulse_sequence(gpulse, dlocs))


In [ ]:
plt.plot(t_axis, K(t_axis), label='K')
plt.plot(t_axis, D(t_axis), label='D')
plt.plot(t_axis, P(t_axis), label='P')
plt.xlabel("time (s)")
plt.ylabel("envelope (V)")
plt.xlim([0, 1])
plt.legend();

Now let's define and integrate the ODE:

In [ ]:
from scipy.integrate import solve_ivp

In [ ]:
def gradfun(t, y, K, D, P):
    eps = eps1 + eps2 * K(t)
    B = beta1 + beta2 * P(t) * 10
    D = delta * D(t)

    xdot = y[1]
    ydot = -eps * y[0] - C * y[0]**2 * y[1] + B * y[1] - D

    return (xdot, ydot)

In [ ]:
soln = solve_ivp(gradfun, (0, 1), (0, 0), args=(K, D, P), first_step=0.5/sr)
print(soln.status, soln.message)

In [ ]:
plt.plot(soln.t, soln.y[0])

In [ ]:
D0 = 10 * delta * 0.06
B0 = beta1 + beta2 * 0.015 
eps0 = eps1 + eps2 * 0.05

print(D0, B0, eps0, eps0 * np.sqrt(B0/C))

In [ ]:
plt.plot(t_axis, (beta1 + beta2 * P(t_axis) * 10)**2)
plt.plot(t_axis, eps1 + eps2 * K(t_axis))